In [ ]:
!export HADOOP_CLASSPATH=/tmp/gcs-connector-hadoop3-latest.jar

In [ ]:
import json
creds={}
with open('/tmp/keys.json', 'r') as f:
  creds = json.load(f)
print(creds)

In [ ]:
from pyspark.sql import SparkSession


spark = SparkSession.builder.appName('GCSFilesRead').config('spark.driver.extraClassPath', '/tmp/gcs-connector-hadoop3-latest.jar').config('fs.gs.auth.service.account.private.key.id', creds['private_key_id']).config('fs.gs.auth.service.account.private.key', creds['private_key']).config('fs.gs.auth.service.account.email', creds['client_email']).getOrCreate()
spark.sparkContext._jsc.addJar('/tmp/gcs-connector-hadoop3-latest.jar')
spark._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
spark._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")


In [ ]:
from pyspark import SparkContext
sc = spark.sparkContext

import random

NUM_SAMPLES = 1000

def inside(p):
 x, y = random.random(), random.random()
 return x*x + y*y < 1

count = sc.parallelize(range(0, NUM_SAMPLES)).filter(inside).count()
pi = 4 * count / NUM_SAMPLES


# calls the function the number of times required to calculate pi based on sample number.

print("Pi is roughly:", pi, ", Number of parallel function calls:", NUM_SAMPLES)

In [ ]:
import os

bucket_name="verizon-dev"
path=f"gs://{bucket_name}/test.csv"

In [ ]:
df = spark.read.option("header",True).csv(path)
df.show()

In [ ]:
path=f"gs://{bucket_name}/myprefix/"
df.write.save(path)

In [ ]:
spark.stop()